In [29]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import  SVC
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix,mean_absolute_error

In [4]:

data=pd.read_csv("D:\ML\ML Assignment\deforestation_dataset.csv")
df=pd.DataFrame(data)
print(df.head)

<bound method NDFrame.head of       Country  Year  Forest_Loss_Area_km2  Tree_Cover_Loss_percent  \
0   Indonesia  1971                   560                 8.929641   
1      Brazil  1927                  3303                 4.638441   
2      Russia  1961                  4466                 4.679313   
3   Australia  1967                  3658                 1.535528   
4   Australia  1987                  2682                 8.035841   
..        ...   ...                   ...                      ...   
95  Australia  2005                  1809                 1.544935   
96  Australia  2001                  2017                 4.264310   
97  Australia  1981                  3960                 2.316448   
98  Australia  1973                  2466                 6.562127   
99      India  1974                   875                 8.943383   

    CO2_Emission_mt  Rainfall_mm  Population  GDP_Billion_USD  \
0               304  1635.715350    86759840      2551.805035   

In [6]:
# Check data types, null values, and column info
print(df.info())

# Summary statistics (helps catch outliers or inconsistent values)
print(df.describe())
# Count total missing values per column
print(df.isnull().sum())
print(df.duplicated().sum())
print(df["Year"].min(), df["Year"].max())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 14 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          100 non-null    object 
 1   Year                             100 non-null    int64  
 2   Forest_Loss_Area_km2             100 non-null    int64  
 3   Tree_Cover_Loss_percent          100 non-null    float64
 4   CO2_Emission_mt                  100 non-null    int64  
 5   Rainfall_mm                      100 non-null    float64
 6   Population                       100 non-null    int64  
 7   GDP_Billion_USD                  100 non-null    float64
 8   Agriculture_Land_Percent         100 non-null    float64
 9   Deforestation_Policy_Strictness  100 non-null    int64  
 10  Corruption_Index                 100 non-null    float64
 11  International_Aid_Million_USD    100 non-null    int64  
 12  Illegal_Lumbering_Incid

In [18]:
df.select_dtypes(include='object').columns
print(df.columns)
print(df["Country_encoded"])


Index(['Year', 'Forest_Loss_Area_km2', 'Tree_Cover_Loss_percent',
       'CO2_Emission_mt', 'Rainfall_mm', 'Population', 'GDP_Billion_USD',
       'Agriculture_Land_Percent', 'Deforestation_Policy_Strictness',
       'Corruption_Index', 'International_Aid_Million_USD',
       'Illegal_Lumbering_Incidents', 'Protected_Areas_Percent',
       'Country_encoded'],
      dtype='object')
0     3
1     1
2     4
3     0
4     0
     ..
95    0
96    0
97    0
98    0
99    2
Name: Country_encoded, Length: 100, dtype: int64


In [22]:
from sklearn.preprocessing import StandardScaler
df.select_dtypes(include='int').columns
withot_scaled=df[["CO2_Emission_mt","Population","International_Aid_Million_USD"]]
scaler = StandardScaler()
x_scaled = scaler.fit_transform(withot_scaled)


In [28]:
#Split Data into Training and Testing Sets
print(df.columns)
y=df["Forest_Loss_Area_km2"]
x_train, x_test, y_train, y_test = train_test_split(
    x_scaled, y, test_size=0.2, random_state=42
)
svm_model = SVC(kernel='rbf')  # SVM for classification
svm_model.fit(x_train, y_train)
y_pred = svm_model.predict(x_test)

Index(['Year', 'Forest_Loss_Area_km2', 'Tree_Cover_Loss_percent',
       'CO2_Emission_mt', 'Rainfall_mm', 'Population', 'GDP_Billion_USD',
       'Agriculture_Land_Percent', 'Deforestation_Policy_Strictness',
       'Corruption_Index', 'International_Aid_Million_USD',
       'Illegal_Lumbering_Incidents', 'Protected_Areas_Percent',
       'Country_encoded'],
      dtype='object')


In [30]:
# Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred)

# Mean Squared Error
mse = mean_squared_error(y_test, y_pred)

# R-squared
r2 = r2_score(y_test, y_pred)

print("Model Evaluation Metrics:")
print(f"MAE  : {mae:.3f}")
print(f"MSE  : {mse:.3f}")
print(f"R2   : {r2:.3f}")


Model Evaluation Metrics:
MAE  : 1244.350
MSE  : 2745328.350
R2   : -1.568
